%md
###Import Dependancy Notebooks and library

In [0]:
# Importing libraries
import os
import re
import time
import json
import requests
import traceback
from datetime import datetime, date, timedelta
from functools import reduce
from pytz import timezone
import pandas as pd
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import _parse_datatype_string, IntegerType, StringType, TimestampType, FloatType
from pyspark.sql.functions import (col, isnull, when, concat, abs, lit, lower, regexp_replace, regexp_extract, trim, current_timestamp, desc, row_number, size, replace, to_date, month, year, last, coalesce, expr)
from delta.tables import DeltaTable
import datetime
start_time = datetime.datetime.now()

### Utils

In [0]:
%run "/Workspace/data_engineering/utils/nb_pmrs_utils_config"

In [0]:

%run "/Workspace/data_engineering/utils/nb_pmrs_utils_audit_log_func"


###Widgets

In [0]:
debug_flag          = dbutils.jobs.taskValues.get(taskKey="Raw_to_Bronze_staging_ingestion", key="debug_flag")
uc_catalog_name     = dbutils.jobs.taskValues.get(taskKey="Raw_to_Bronze_staging_ingestion", key="uc_catalog_name")
object_owner_spn    = dbutils.jobs.taskValues.get(taskKey="Raw_to_Bronze_staging_ingestion", key="object_owner_spn")
external_location   = dbutils.jobs.taskValues.get(taskKey="Raw_to_Bronze_staging_ingestion", key="external_location")
username            = dbutils.jobs.taskValues.get(taskKey="Raw_to_Bronze_staging_ingestion", key="username")
data_feed            = dbutils.jobs.taskValues.get(taskKey="Raw_to_Bronze_staging_ingestion", key="data_feed")
log_id              = dbutils.jobs.taskValues.get(taskKey="Raw_to_Bronze_staging_ingestion", key="log_id")


dbutils.jobs.taskValues.set("external_location", external_location)
dbutils.jobs.taskValues.set("uc_catalog_name", uc_catalog_name)
dbutils.jobs.taskValues.set("object_owner_spn", object_owner_spn)
dbutils.jobs.taskValues.set("debug_flag", debug_flag)
dbutils.jobs.taskValues.set("username", username)
dbutils.jobs.taskValues.set("data_feed", data_feed)
dbutils.jobs.taskValues.set("log_id", log_id)

In [0]:

dbutils.widgets.removeAll()

# Create text widgets for various master data feeds and retrieve their values

dbutils.widgets.text("product_data_feed","master_data_product_category")
product_data_feed       = dbutils.widgets.get("product_data_feed")

dbutils.widgets.text("flow_data_feed","master_data_flow")
flow_data_feed       = dbutils.widgets.get("flow_data_feed")

dbutils.widgets.text("measure_data_feed","master_data_measure")
measure_data_feed       = dbutils.widgets.get("measure_data_feed")

dbutils.widgets.text("geography_data_feed","master_data_geography")
geography_data_feed       = dbutils.widgets.get("geography_data_feed")

dbutils.widgets.text("brand_data_feed","master_data_brand")
brand_data_feed       = dbutils.widgets.get("brand_data_feed")

dbutils.widgets.text("brand_pos_data_feed","master_data_brand_position")
brand_pos_data_feed       = dbutils.widgets.get("brand_pos_data_feed")

dbutils.widgets.text("manufacturer_data_feed","master_data_manufacturer")
manufacturer_data_feed       = dbutils.widgets.get("manufacturer_data_feed")

dbutils.widgets.text("debug_flag","1")
debug_flag      = dbutils.widgets.get("debug_flag")
dbutils.jobs.taskValues.set(key = "debug_flag", value = debug_flag)

In [0]:
try:
    # Get the absolute path
    absolute_path = return_absolute_path(external_location)

    # Get the current notebook name
    current_nb_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get().split("/")[-1]

    # Get feed ID, module ID, and feed type ID
    feed_id, module_id, feed_type_id = get_audit_feed_info(data_feed, uc_catalog_name, current_nb_name)

    # Get status IDs
    status_failure_id, status_success_id, status_running_id = get_status_ids(uc_catalog_name)

    # Get run ID and job ID
    run_id = int(dbutils.notebook.entry_point.getDbutils().notebook().getContext().parentRunId().get())
    job_id = int(dbutils.notebook.entry_point.getDbutils().notebook().getContext().jobId().get())

    run_id = int(log_id)

    # Debugging information
    if debug_flag == "1":
        print(f"run_id : {run_id}, job_id : {job_id}, log_id : {log_id}\n")
        print(f"absolute_path        : {absolute_path}")
        print(f"data_feed            : {data_feed}")
        print(f"external_location    : {external_location}")
        print(f"notebook_name        : {current_nb_name}")
        print(f"object_owner_spn     : {object_owner_spn}")
        print(f"uc_catalog_name      : {uc_catalog_name}")
        print(f"username             : {username}")

except Exception as e:
    error = str(e).replace("'", "")
    raise Exception(f"Error occurred: {error}")

### Add Pipeline Entry To AuditLog

In [0]:
# Insert a job record into the job table
counter = 1
log_value = run_id + 2
detail_log_value = log_value * 10
 
# Insert job log
insert_job_log(log_value, job_id, run_id, username, "Market Bronze to Silver Fact GMI Recon", datetime.now(), None, status_running_id, feed_type_id, feed_id, module_id, 0, "", "", uc_catalog_name)
 
# Insert job detail log
insert_job_detail_log(int(detail_log_value + counter), log_value, run_id, username, "Bronze to silver recon for Fact GMI Snapshot", datetime.now(), status_running_id, feed_type_id, feed_id, module_id, 0, "", uc_catalog_name, "")
 
# Debugging output
if debug_flag == "1":
    print('log_id:', log_id)

###Main Orchestration

In [0]:
try:
    # ------------------Reading the Ingestion_Configuration_sheet-------------------#

    # Get the ingestion sheet data
    
    param = get_param_data(uc_catalog_name)

    # Fetch specific configuration
    process_feed_df                      = fetch_config(param, data_feed)
    stream = process_feed_df.select("stream").first()[0]

    config_data = get_config_data(data_feed,uc_catalog_name,stream)

    catalog_name                         = uc_catalog_name
    delta_db_staging                     = process_feed_df.select("delta_db_staging").first()[0]
    delta_table_staging                  = process_feed_df.select("delta_table_staging").first()[0]

        # json values
    delta_path_silver                    = json.loads(process_feed_df.select("delta_path_silver").first()[0].replace("'",'"'))
    delta_table_silver                   = json.loads(process_feed_df.select("delta_table_silver").first()[0].replace("'",'"'))['fact_gmi']
    delta_db_silver                      = json.loads(process_feed_df.select("delta_db_silver").first()[0].replace("'",'"'))['fact_gmi']

    rename_columns = config_data.get("rename_cols_silver",{})
    if rename_columns:
        rename = True
    else:
        rename = False

    if debug_flag == "1":

        print('data_feed                        :', data_feed)
        print('catalog_name                     :', catalog_name)
        print('delta_db_staging                 :', delta_db_staging)
        print('delta_table_staging              :', delta_table_staging)
        print('delta_table_silver               :', delta_table_silver)
        print('delta_db_silver                  :', delta_db_silver)

except Exception as e:
    error = str(e).replace("'", "")
    error_code = type(e).__name__
    counter = counter + 1
    update_job(job_id, run_id, module_id, feed_type_id, feed_id, status_failure_id,uc_catalog_name)
    update_job_log(log_value,status_failure_id, job_id, run_id, feed_type_id, feed_id, module_id,uc_catalog_name)
    insert_job_detail_log(int(detail_log_value + counter) , log_value, run_id, username, "Fetching config file columns for the silver ingestion failure", datetime.now(), status_running_id, feed_type_id, feed_id, module_id, 0, error_code,uc_catalog_name, error)
    raise e

In [0]:
try:
    # Bronze raw data
    df_bronze = spark.table(f"{uc_catalog_name}.{delta_db_staging}.{delta_table_staging}")

    # Bronze transformation for period to get month_sk
    df_bronze =  df_bronze.withColumn("month_sk",when((day(current_date()) <= 31) & (day(current_date()) > 21), concat(year(add_months(current_date(), -1)),
    lpad(month(add_months(current_date(), -1)).cast("string"),2,"0")))
    .otherwise(concat(year(add_months(current_date(), -2)),
    lpad(month(add_months(current_date(), -2)).cast("string"),2,"0"))))
                                    
    df_bronze = df_bronze.withColumn("year", col('period').substr(-4,4)).withColumn("month",
                when(col('period').contains("4WEEK1-") | col('period').contains("MTH1-"), '01').
                when(col('period').contains("4WEEK2") | col('period').contains("MTH2"), '02').
                when(col('period').contains("4WEEK3") | col('period').contains("MTH3"), '03').
                when(col('period').contains("4WEEK4") | col('period').contains("MTH4"), '04').
                when(col('period').contains("4WEEK5") | col('period').contains("MTH5"), '05').
                when(col('period').contains("4WEEK6") | col('period').contains("MTH6"), '06').
                when(col('period').contains("4WEEK7") | col('period').contains("MTH7"), '07').
                when(col('period').contains("4WEEK8") | col('period').contains("MTH8"), '08').
                when(col('period').contains("4WEEK9") | col('period').contains("MTH9"), '09').
                when(col('period').contains("4WEEK10") | col('period').contains("MTH10"), '10').
                when(col('period').contains("4WEEK11") | col('period').contains("4WEEK12") | col('period').contains("MTH11"), '11').
                when(col('period').contains("4WEEK13") | col('period').contains("MTH12"), '12').
                otherwise(col('period')))

    df_bronze = df_bronze.withColumn("month_sk",
                                when(col('period').contains("4WEEK") | col('period').contains("MTH"), concat(col('year'),col('month')))
                                .otherwise(col('month_sk')))
    df_fact_trans = df_bronze
    df_fact_trans = df_fact_trans.withColumn("month_sk",
                                when(col('period').contains("4WEEK") | col('period').contains("MTH"), concat(col('year'),col('month')))
                                .otherwise(col('month_sk')))
    
    # Bronze col addition for flow description comparison logic
    df_fact_trans = df_fact_trans.withColumn("Sales_Period_Group_ID",when (col("period").contains("4WEEK"), "4W")
                                .when (col("period").contains("MTH"), "MON")
                                .otherwise(col("period")))

    df_fact = df_fact_trans.filter(col("Sales_Period_Group_ID").isin(['L12W','L12W-1','L12W-1Y','L12W-2','L12W-2Y','L12W-3Y','L12W-4Y','MAT','MAT-1','MAT-2','MAT-3','MAT-4','YTD','YTD-1','YTD-2','YTD-3','YTD-4','L4W','L4W-1Y','L4W-1','L4W-2',"4W","MON"]) | col("Sales_Period_Group_ID").rlike('FY-.*'))
    df_bronze = df_fact

except Exception as e:
    error = str(e).replace("'", "")
    error_code = type(e).__name__
    counter = counter + 1
    update_job(job_id, run_id, module_id, feed_type_id, feed_id, status_failure_id,uc_catalog_name)
    update_job_log(log_value,status_failure_id, job_id, run_id, feed_type_id, feed_id, module_id,uc_catalog_name)
    insert_job_detail_log(int(detail_log_value + counter) , log_value, run_id, username, "Failed to read or transform bronze data", datetime.now(), status_running_id, feed_type_id, feed_id, module_id, 0, error_code,uc_catalog_name, error)
    raise e

In [0]:
try:
    # Key creation for bronze, final bronze recon dataframe

    df_bronze_recon = df_bronze.select(concat_ws('_', 'country_code', 'product_form_code', 'brand_code', 'local_manufacturer_code','Sales_Period_Group_ID','month_sk').alias('b_key'),'cy_sales_value', 'cy_sales_volume')

    df_bronze_recon = df_bronze_recon.groupBy('b_key').sum('cy_sales_value', 'cy_sales_volume').withColumnRenamed('sum(cy_sales_value)', 'Value').withColumnRenamed('sum(cy_sales_volume)', 'Volume')

except Exception as e:
    error = str(e).replace("'", "")
    error_code = type(e).__name__
    counter = counter + 1
    update_job(job_id, run_id, module_id, feed_type_id, feed_id, status_failure_id,uc_catalog_name)
    update_job_log(log_value,status_failure_id, job_id, run_id, feed_type_id, feed_id, module_id,uc_catalog_name)
    insert_job_detail_log(int(detail_log_value + counter) , log_value, run_id, username, "Failed to create bronze and silver keys", datetime.now(), status_running_id, feed_type_id, feed_id, module_id, 0, error_code,uc_catalog_name, error)
    raise e

In [0]:
try:
    # Silver transformed data
    df_silver = spark.table(f"{uc_catalog_name}.{delta_db_silver}.{delta_table_silver}")

except Exception as e:
    error = str(e).replace("'", "")
    error_code = type(e).__name__
    counter = counter + 1
    update_job(job_id, run_id, module_id, feed_type_id, feed_id, status_failure_id,uc_catalog_name)
    update_job_log(log_value,status_failure_id, job_id, run_id, feed_type_id, feed_id, module_id,uc_catalog_name)
    insert_job_detail_log(int(detail_log_value + counter) , log_value, run_id, username, "Failed to read silver data", datetime.now(), status_running_id, feed_type_id, feed_id, module_id, 0, error_code,uc_catalog_name, error)
    raise e

In [0]:
try:
    # Get dimension tables and join with silver to get codes

    df_product = spark.table(f"{uc_catalog_name}.silver_master_data.product")
    df_brand = spark.table(f"{uc_catalog_name}.silver_master_data.brand")
    df_flow = spark.table(f"{uc_catalog_name}.silver_master_data.flow")
    df_measure = spark.table(f"{uc_catalog_name}.silver_master_data.measure")
    df_manufacturer = spark.table(f"{uc_catalog_name}.silver_master_data.manufacturer")
    df_calendar = spark.table(f"{uc_catalog_name}.silver_master_data.calendar")
    df_geography = spark.table(f"{uc_catalog_name}.silver_master_data.geography")

    df_silver_recon = df_silver.join(df_product, df_silver.product_sk == df_product.product_sk, how='inner') \
                            .join(df_brand, df_silver.brand_sk == df_brand.brand_sk, how='inner') \
                            .join(df_flow, df_silver.flow_sk == df_flow.flow_sk, how='inner') \
                            .join(df_measure, df_silver.measure_sk == df_measure.measure_sk, how='inner') \
                            .join(df_geography, df_silver.geography_sk == df_geography.geography_sk, how='inner') \
                            .join(df_manufacturer, df_silver.manufacturer_sk == df_manufacturer.manufacturer_sk, how='inner') \
                            .select('mu_iso_country_code',
                                    'product_form_code',
                                    'corporate_brand_code',
                                    'local_manufacturer_code',
                                    'month_sk',
                                    'flow_description',
                                    'measure_description',
                                    'value'
                                    )

except Exception as e:
    error = str(e).replace("'", "")
    error_code = type(e).__name__
    counter = counter + 1
    update_job(job_id, run_id, module_id, feed_type_id, feed_id, status_failure_id,uc_catalog_name)
    update_job_log(log_value,status_failure_id, job_id, run_id, feed_type_id, feed_id, module_id,uc_catalog_name)
    insert_job_detail_log(int(detail_log_value + counter) , log_value, run_id, username, "Failed to read master data or join with silver data", datetime.now(), status_running_id, feed_type_id, feed_id, module_id, 0, error_code,uc_catalog_name, error)
    raise e

In [0]:
try:   
    # Silver recon dataframe
    df_silver_recon = df_silver_recon.groupBy('mu_iso_country_code',
        'product_form_code',
        'corporate_brand_code',
        'local_manufacturer_code',
        'flow_description',
        'month_sk').pivot('measure_description').agg(sum('value')).select(
            concat_ws('_', 'mu_iso_country_code', 'product_form_code', 'corporate_brand_code', 'local_manufacturer_code', 'flow_description', 'month_sk').alias('s_key'),'Value','Volume')

    # From original logic, this matches scale of bronze and silver values
    df_silver_recon = df_silver_recon.withColumn("Value", col('Value') * 1000)
    df_silver_recon = df_silver_recon.withColumn("Volume", col('Volume') * 1000)

    # Rename columns to differentiate between bronze and silver data
    df_silver_recon = df_silver_recon.withColumnRenamed("Value","Value_s")
    df_silver_recon = df_silver_recon.withColumnRenamed("Volume","Volume_s")
    df_bronze_recon = df_bronze_recon.withColumnRenamed("Value","Value_b")
    df_bronze_recon = df_bronze_recon.withColumnRenamed("Volume","Volume_b")
    
except Exception as e:
    error = str(e).replace("'", "")
    error_code = type(e).__name__
    counter = counter + 1
    update_job(job_id, run_id, module_id, feed_type_id, feed_id, status_failure_id,uc_catalog_name)
    update_job_log(log_value,status_failure_id, job_id, run_id, feed_type_id, feed_id, module_id,uc_catalog_name)
    insert_job_detail_log(int(detail_log_value + counter) , log_value, run_id, username, "Failed during creation of final recon dataframes", datetime.now(), status_running_id, feed_type_id, feed_id, module_id, 0, error_code,uc_catalog_name, error)
    raise e

In [0]:
try:
    # Join bronze and silver for comparing value and volume

    df_temp = df_bronze_recon.join(df_silver_recon, df_bronze_recon.b_key == df_silver_recon.s_key, how='inner')

    df_temp = df_temp.withColumn("Value_b", round(col('Value_b'),3))
    df_temp = df_temp.withColumn("Value_s", round(col('Value_s'),3))
    df_temp = df_temp.withColumn("Volume_b", round(col('Volume_b'),3))
    df_temp = df_temp.withColumn("Volume_s", round(col('Volume_s'),3))

    df_temp = df_temp.withColumn("Value_diff", round(col("Value_s") - col("Value_b"),0))
    df_temp = df_temp.withColumn("Volume_diff", round(col("Volume_s") - col("Volume_b"),0))

    check = df_temp.agg(sum(abs(col("Value_diff"))), sum(abs(col("Volume_diff")))).collect()[0]
    for i in check:
        if i != 0:
            raise Exception("Bronze and Silver values do not match")

except Exception as e:
    error = str(e).replace("'", "")
    error_code = type(e).__name__
    counter = counter + 1
    update_job(job_id, run_id, module_id, feed_type_id, feed_id, status_failure_id,uc_catalog_name)
    update_job_log(log_value,status_failure_id, job_id, run_id, feed_type_id, feed_id, module_id,uc_catalog_name)
    insert_job_detail_log(int(detail_log_value + counter) , log_value, run_id, username, "Failed to create comparison dataframe", datetime.now(), status_running_id, feed_type_id, feed_id, module_id, 0, error_code,uc_catalog_name, error)
    raise e